In [2]:
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import random
import pdb

### Make a list of the urls of the categories in the forum

In [3]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://curltalk.naturallycurly.com/', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser')
link_list_forum_top = []
for link in soup.find_all('a'):
    link_list_forum_top.append(link.get('href'))
categories = [s for s in link_list_forum_top if (("categories" in s) and ('https' in s))]
categories[0:4]

['https://curltalk.naturallycurly.com/categories/announcements',
 'https://curltalk.naturallycurly.com/categories/newbies-info',
 'https://curltalk.naturallycurly.com/categories/introductions',
 'https://curltalk.naturallycurly.com/categories/curlmart-news']

#### Get the discussion topic urls for the first x pages of the category

In [4]:
# %%time

link_listdiscussion = []
for i in range(11, 21):
    url = f'{categories[5]}/p{i}'
    print(url)
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
     
    # Check status code for an appropriate response fromt the API
    status = html_page.status_code
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        break
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    page1_of_discussion = []
    time.sleep(random.randint(20,45))
    for link in soup.find_all('a'):
        link_listdiscussion.append(link.get('href'))
        topics = [s for s in link_listdiscussion if (("/discussion" in s) and ('https' in s))]

print(len(topics))     
# Loop through all the topics found for each catagory        
list_for_mongo = []
count = 1
for topic in topics:
    url2 = topic
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page2 = requests.get(url2, headers = {'User-Agent': user_agent} )
    
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        break
        
    soup2 = BeautifulSoup(html_page2.content, 'html.parser')
    signatures = soup2.find_all('div', class_="Signature UserSignature userContent")
    
    for i in range(0,len(signatures)):
        sig = {}
        sig['signature'] = soup2.find_all('div', class_="Signature UserSignature userContent")[i].get_text()
        list_for_mongo.append(sig)
    time.sleep(random.randint(60,90))
    print(count)
    count += 1
    

https://curltalk.naturallycurly.com/categories/c2/p11
https://curltalk.naturallycurly.com/categories/c2/p12
https://curltalk.naturallycurly.com/categories/c2/p13
https://curltalk.naturallycurly.com/categories/c2/p14
https://curltalk.naturallycurly.com/categories/c2/p15
https://curltalk.naturallycurly.com/categories/c2/p16
https://curltalk.naturallycurly.com/categories/c2/p17
https://curltalk.naturallycurly.com/categories/c2/p18
https://curltalk.naturallycurly.com/categories/c2/p19
https://curltalk.naturallycurly.com/categories/c2/p20
300
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141


ConnectionError: HTTPSConnectionPool(host='curltalk.naturallycurly.com', port=443): Max retries exceeded with url: /discussion/181774/devacut-on-2b-c-hair (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x108b53c18>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [5]:
list_for_mongo

[{'signature': 'Low-poo: Alaffia Coffee & Shea Revitalizing Shampoo for Fine/Thin Hair (summer), Alaffia Coconut & Shea Hydrating ShampooClarify: Kinky Curly Come CleanRinse-out: Alaffia Coffee & Shea Revitalizing Conditioner for Fine/Thin Hair (summer), Alaffia Everyday Coconut Ultra Hydrating ConditionerDT: Camille Rose Naturals Algae Renew Deep ConditionerLeave-in:\xa0As I Am Leave-inStyler:\xa0Lily of the Desert 99% Aloe Vera GellyTreatment:\xa0Daily tea spray; Coconut oil overnight treatment; Citric acid rinse for hard water; 2-step Henndigo w/ amla to cover gray\n2C, fine, low porosity, low/medium density, protein sensitive\nLooking for styling products and tools\n\niHerb Code KQS149 for 5% off your order'},
 {'signature': '__\n2C-3A Fine, medium density, long\nModified CG Method / Natural Only since 2015\n:thumbright:Canadian Mama!'},
 {'signature': 'Gretchen\nNaturallyCurly.com co-founder\n3A\n\nYou are beautiful!'},
 {'signature': '2b, 2c,\nHigh Porosity - Very Fine Hair\n\nLo

### Create a mongoDB and have data stored from scrape 

In [6]:
# DB creation 
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [7]:
entry2 = curly_collection.insert_many(list_for_mongo)

In [8]:
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])
    #print(x)
print(len(test))    

15624


In [9]:
test_set = set(test)

In [10]:
# Number of unique entries so far
len(test_set)
#1406

1406

In [11]:
query_2 = curly_collection.find({})
test2 = []
for x in query_2:
    test2.append(x['signature'])
    #print(x)
print(len(test2))  

15624


In [13]:
len(set(test2))

1406